In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.tree import DecisionTreeClassifier, plot_tree

import acquire
import prepare

# Titanic

Using the titanic data, in your classification-exercises repository, create a notebook, model.ipynb where you will do the following:

In [34]:
titanic_train, titanic_validate, titanic_test = prepare.train_validate_test_split(prepare.prep_titanic(acquire.get_titanic_data()), 'survived')

What is your baseline prediction? 

In [35]:
titanic_train.survived.value_counts()

0    307
1    191
Name: survived, dtype: int64

In [36]:
titanic_train['baseline_prediction'] = titanic_train.survived.value_counts().idxmax()

In [37]:
titanic_train.head()

,survived,sex,sibsp,parch,fare,class,embark_town,alone,sex_male,class_Second,class_Third,embark_town_Queenstown,embark_town_Southampton,baseline_prediction
301,1,male,2,0,23.2500,Third,Queenstown,0,1,0,1,1,0,0
290,1,female,0,0,78.8500,First,Southampton,1,0,0,0,0,1,0
779,1,female,0,1,211.3375,First,Southampton,0,0,0,0,0,1,0
356,1,female,0,1,55.0000,First,Southampton,0,0,0,0,0,1,0
147,0,female,2,2,34.3750,Third,Southampton,0,0,0,1,0,1,0


What is your baseline accuracy? *remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.*

In [38]:
baseline_accuracy = metrics.accuracy_score(y_true=titanic_train.survived, y_pred=titanic_train.baseline_prediction)

print(f'Baseline accuracy: {baseline_accuracy:%}')

Baseline accuracy: 61.646586%


Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [39]:
titanic_train.head()

,survived,sex,sibsp,parch,fare,class,embark_town,alone,sex_male,class_Second,class_Third,embark_town_Queenstown,embark_town_Southampton,baseline_prediction
301,1,male,2,0,23.2500,Third,Queenstown,0,1,0,1,1,0,0
290,1,female,0,0,78.8500,First,Southampton,1,0,0,0,0,1,0
779,1,female,0,1,211.3375,First,Southampton,0,0,0,0,0,1,0
356,1,female,0,1,55.0000,First,Southampton,0,0,0,0,0,1,0
147,0,female,2,2,34.3750,Third,Southampton,0,0,0,1,0,1,0


In [40]:
drop_columns = list(titanic_train.select_dtypes(include='object').columns)

titanic_train.drop(columns=drop_columns, inplace=True)

In [41]:
X_train = titanic_train.drop(columns=['survived'])
y_train = titanic_train.survived

In [42]:
clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [56]:
y_pred = clf.predict(X_train)

Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [ ]:
#model score
clf.score(X_train, y_train)

In [41]:
X_train = titanic_train.drop(columns=['survived'])
y_train = titanic_train.survived

In [42]:
clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [56]:
y_pred = clf.predict(X_train)

Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [ ]:
#model score
clf.score(X_train, y_train)

In [47]:
#confusion matrix
metrics.confusion_matrix(y_train, y_pred)

array([[297,  10],
       [ 21, 170]])

In [49]:
#classification report
print(metrics.classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       307
           1       0.94      0.89      0.92       191

    accuracy                           0.94       498
   macro avg       0.94      0.93      0.93       498
weighted avg       0.94      0.94      0.94       498



In [51]:
tn = 297
tp = 170
fp = 10
fn = 21

tpr = recall = tp/ (tp + fn)
print(f'true positive rate: {tpr}')
fpr = fp / (fp + tn)
print(f'false positive rate: {fpr}')
tnr = tn / (tn + fp)
print(f'true negative rate: {tnr}')
fnr = fn / (fn + tp)
print(f'false negative rate: {fnr}')

true positive rate: 0.8900523560209425
false positive rate: 0.03257328990228013
true negative rate: 0.9674267100977199
false negative rate: 0.1099476439790576


### Compute: 
- Accuracy: 0.94
- precision: 0.94
- recall: 0.89
- f1-score:
- support:

Run through steps 2-4 using a different max_depth value.

In [52]:
clf = DecisionTreeClassifier(max_depth = 6)

clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6)

In [57]:
y_pred = clf.predict(X_train)

In [58]:
#model score
clf.score(X_train, y_train)

0.8614457831325302

In [59]:
#confusion matrix
metrics.confusion_matrix(y_train, y_pred)

array([[285,  22],
       [ 47, 144]])

In [60]:
#classification report
print(metrics.classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.93      0.89       307
           1       0.87      0.75      0.81       191

    accuracy                           0.86       498
   macro avg       0.86      0.84      0.85       498
weighted avg       0.86      0.86      0.86       498



In [61]:
tn = 285
tp = 144
fp = 22
fn = 47

tpr = recall = tp/ (tp + fn)
print(f'true positive rate: {tpr}')
fpr = fp / (fp + tn)
print(f'false positive rate: {fpr}')
tnr = tn / (tn + fp)
print(f'true negative rate: {tnr}')
fnr = fn / (fn + tp)
print(f'false negative rate: {fnr}')

true positive rate: 0.7539267015706806
false positive rate: 0.07166123778501629
true negative rate: 0.9283387622149837
false negative rate: 0.24607329842931938


###### Compute: 
- Accuracy: 0.86
- precision: 0.86
- recall: 0.93
- f1-score:
- support:

Which model performs better on your in-sample data?

Which model performs best on your out-of-sample data, the validate set?

## Telco

In [62]:
telco_train, telco_validate, telco_test = prepare.train_validate_test_split(prepare.prep_telco(acquire.get_telco_data()), 'churn')

/Users/fostermark/codeup-data-science/classification-exercises/prepare.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.total_charges = df.total_charges.astype(float)


In [63]:
telco_train.churn_Yes.value_counts()

0    2891
1    1046
Name: churn_Yes, dtype: int64

In [64]:
telco_train['baseline_prediction'] = telco_train.churn_Yes.value_counts().idxmax()

In [65]:
telco_train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check,baseline_prediction
4491,Male,0,No,No,48,Yes,No,No,Yes,Yes,...,0,0,0,1,0,0,1,0,0,0
2346,Female,0,Yes,Yes,58,Yes,No,No internet service,No internet service,No internet service,...,1,0,0,1,0,1,0,0,1,0
770,Female,0,No,Yes,44,Yes,Yes,No,Yes,No,...,1,0,1,0,0,0,0,0,1,0
5155,Male,0,Yes,No,10,Yes,Yes,No,No,No,...,1,1,0,0,1,0,0,1,0,0
189,Female,0,No,Yes,11,No,No phone service,Yes,No,No,...,1,0,1,0,0,0,0,0,1,0


In [66]:
drop_columns = list(telco_train.select_dtypes(include='object').columns)

telco_train.drop(columns=drop_columns, inplace=True)

In [67]:
telco_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3937 entries, 4491 to 3781
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   senior_citizen                         3937 non-null   int64  
 1   tenure                                 3937 non-null   int64  
 2   monthly_charges                        3937 non-null   float64
 3   total_charges                          3937 non-null   float64
 4   gender_Male                            3937 non-null   uint8  
 5   partner_Yes                            3937 non-null   uint8  
 6   dependents_Yes                         3937 non-null   uint8  
 7   phone_service_Yes                      3937 non-null   uint8  
 8   multiple_lines_No phone service        3937 non-null   uint8  
 9   multiple_lines_Yes                     3937 non-null   uint8  
 10  online_security_No internet service    3937 non-null   uint8  
 11  o

In [68]:
X_train = telco_train.drop(columns=['churn_Yes'])
y_train = telco_train.churn_Yes

In [69]:
clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [70]:
y_pred = clf.predict(X_train)

In [71]:
#model score
clf.score(X_train, y_train)

0.9982219964439929

In [72]:
#confusion matrix
metrics.confusion_matrix(y_train, y_pred)

array([[2891,    0],
       [   7, 1039]])

In [73]:
#classification report
print(metrics.classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2891
           1       1.00      0.99      1.00      1046

    accuracy                           1.00      3937
   macro avg       1.00      1.00      1.00      3937
weighted avg       1.00      1.00      1.00      3937



In [75]:
tn = 2891
fn = 7
fp = 0
tp = 1039



tpr = recall = tp/ (tp + fn)
print(f'true positive rate: {tpr}')
fpr = fp / (fp + tn)
print(f'false positive rate: {fpr}')
tnr = tn / (tn + fp)
print(f'true negative rate: {tnr}')
fnr = fn / (fn + tp)
print(f'false negative rate: {fnr}')

true positive rate: 0.9933078393881453
false positive rate: 0.0
true negative rate: 1.0
false negative rate: 0.006692160611854685


In [76]:
clf = DecisionTreeClassifier(max_depth = 3)

clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3)

In [77]:
y_pred = clf.predict(X_train)

In [78]:
#model score
clf.score(X_train, y_train)

0.7813055626111253

In [79]:
#confusion matrix
metrics.confusion_matrix(y_train, y_pred)

array([[2704,  187],
       [ 674,  372]])

In [80]:
#classification report
print(metrics.classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.94      0.86      2891
           1       0.67      0.36      0.46      1046

    accuracy                           0.78      3937
   macro avg       0.73      0.65      0.66      3937
weighted avg       0.76      0.78      0.76      3937

